**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.stats import norm
from scipy.optimize import minimize
from IPython.display import HTML
from IPython.utils.capture import capture_output
from matplotlib.animation import FuncAnimation

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/u8u7vcwy3sosbar/titanic.zip?dl=1", directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

# fix the seed of the random generator
# np.random.seed(4)

# GP-based Bayes opti
def random_point(lbound=0, ubound=5):
    return np.random.uniform(0, 5)
    
def next_point(gp, ybest, acquisition_func):
    x = random_point()
    
    best_score = np.inf
    num_restarts = 10
    
    for i in range(num_restarts):
        res = minimize(lambda xx: -acquisition_func(gp,
                           xx.reshape([-1, 1]), ybest).item(),
                       random_point(),
                       method='L-BFGS-B',
                       bounds=[[0, 5]])

        if res.fun < best_score:
            best_score = res.fun
            x = res.x

    return x

# plotting
def plot_distro(
    x, gp, intervals=True, ax=None,
    xlabel="x", ylabel="y", return_preds=False
):
    if ax is None:
        ax = plt.gca()

    y_mean, y_std = gp.predict(x.reshape((-1, 1)), return_std=True) 
    y_mean, y_std = y_mean.reshape(-1), y_std.reshape(-1)
    
    ax.plot(x, y_mean, 'k', lw=3, zorder=9)
    
    if intervals:
        ax.fill_between(x, y_mean - 3*y_std, y_mean + 3*y_std,
                         alpha=0.2, color='b')
    
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(-5, 5)
    ax.grid(ls='--')
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    if return_preds:
        return y_mean, y_std
    
def plot_distro_func_data(
    x, gp=None, X=None, Y=None, func=None,
    intervals=True, ax=None, return_preds=False
):
    preds = None

    if ax is None:
        ax = plt.gca()

    if not gp is None:
        preds = plot_distro(
            x, gp, intervals=intervals, ax=ax, return_preds=return_preds
        )
        
    if not func is None:
        ax.plot(x, func(x), '--', linewidth=3)
    
    if not X is None and not Y is None:
        ax.scatter(X, Y, c='r', s=50, zorder=10, edgecolors=(0, 0, 0))
    
    return preds

def bayes_opti_anim(fig, gp, x, func, acquisition_func,
                      X=None, Y=None, num_opti_steps=10, ax=None,
                      save_figures=True, save_plain=True,
                      save_plain_nofunc=True, save_nofunc=True):
    with capture_output() as io:
        if ax is None:
            ax = fig.gca()
            
        ax.clear()
        plot_distro_func_data(x, gp, func=func, ax=ax)
        plt.show()
        if save_figures:
            fig.savefig("output/gp_opti_init.svg",
                        bbox_inches="tight", pad_inches=0)
        yield

        if X is None: X = []
        if Y is None: Y = []

        x0 = random_point()
        xbest = x0
        ybest = func(x0)

        for s in range(num_opti_steps):
            nx = next_point(gp, ybest, acquisition_func)
            ny = func(nx)

            if ny < ybest:
                xbest = nx
                ybest = ny

            X.append(nx)
            Y.append(ny)

            gp.fit(np.reshape(X, (-1, 1)),
                   np.reshape(Y, (-1, 1)))
            
            if save_plain:
                ax.clear()
                plot_distro_func_data(x, gp, func=func, X=X, Y=Y, ax=ax,
                                      intervals=False)
                fig.savefig("output/gp_opti_plain_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)
                
            if save_plain_nofunc:
                ax.clear()
                plot_distro_func_data(x, gp, X=X, Y=Y, ax=ax,
                                      intervals=False)
                fig.savefig("output/gp_opti_plain_nofunc_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)
                
            if save_nofunc:
                ax.clear()
                plot_distro_func_data(x, gp, X=X, Y=Y, ax=ax)
                fig.savefig("output/gp_opti_nofunc_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)

            ax.clear()
            plot_distro_func_data(x, gp, func=func, X=X, Y=Y, ax=ax)

            plt.show()
            if save_figures:
                fig.savefig("output/gp_opti_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)
            yield

        ax.clear()
        x = np.linspace(0, 5, 100)
        plot_distro_func_data(x, gp, func=func, X=X, Y=Y, ax=ax)
        ax.scatter([xbest], [ybest], c='cyan', marker='x', linewidth=5,
                    s=100, zorder=10, edgecolors=(0, 0, 0))
        plt.show()
        if save_figures:
            fig.savefig("output/gp_opti_final.svg",
                        bbox_inches="tight", pad_inches=0)
        yield
        
    print("solution:", xbest, ybest)
    return

## Bayesovská optimalizácia

### Intuícia za bayesovskou optimalizáciou

Existuje veľké množstvo rozličných optimalizačných metód. Niektoré z nich, ako napr. metóda klesajúceho gradientu, sú založené na informácii o gradientoch, niektoré dokonca na deriváciách vyššieho rádu: napr. Newtonova metóda a jej mnohé aproximácie. Iné triedy optimalizačných metód zase stavajú na prehľadávaní a používajú rôzne šikovné triky, aby boli výpočtovo únosné. Ďalšia trieda zase pracuje s populáciami kandidátskych riešení a používa rôzne metaheuristické prístupy ako je napr. napodobňovanie evolúcie či iných prirodzených fenoménov.

Nech si však už zvolíme ktorúkoľvek z týchto metód, existuje trieda úloh, ktoré je mimoriadne náročné vyriešiť v rozumnom čase: úlohy, kde je extrémne náročné vyhodnotiť účelovú funkciu. Nanešťastie takéto úlohy tvoria v praxi nemalú časť optimalizačných úloh. Vyskutujú sa prakticky neustále aj v kontexte strojového učenia, pretože metódy majú hyperparametre, ktoré treba typicky ladiť, ak má metóda disiahnuť optimálne výsledky. Na to, aby sa vyhodnotila vhodnosť určitej sady hyperparametrov, je však typicky potrebné natrénovať od začiatku celý model strojového učenia (čo napr. v prípade hlbokého učenia, môže niekedy trvať dni alebo dokonca týždne) a otestovať ho.

#### Optimalizácia s náhradnou funkciou

V takých prípadoch je nevyhnutné zabezpečiť, aby sme boli z každého vyhodnotenia účelovej funkcie schopní získať čo najviac informácie. Na tento by sme mohli použiť body, v ktorých už bola účelová funkcia vyhodnotená, a zostrojiť pomocou nich **náhradný model**  (surrogate model): funkciu, ktorú by sme potom mohli optimalizovať namiesto pôvodnej účelovej funkciu. Toto je to, čo robia metódy **optimalizácie s náhradnou funkciou**  (surrogate optimization methods).

#### Bayesovský prístup ku optimalizácii s náhradnou funkciou

Je tu ale ďalšia otázka. Ako vlastne vyberieme body, v ktorých účelovú funkciu vyhodnotníme, aby sme získali dáta na konštrukciu náhradného modelu? Mohli by sme azda jednoducho zvoliť zopár rovnomerne náhodných vzoriek z definičného oboru účelovej funkcie a použiť tie.

Náš náhradný model však bude len taký dobrý, ako vzorky, ktoré sme si náhodne zvolili. Môže byť stále príliš nákladné zostrojiť dostatočne dobrý náhradný model a v tom prípade sa môže stať, že proces optimalizácie nájde spôsoby, ako využiť chyby modelu, aby získal vynikajúce, ale nerealistické výsledky.

Našťastie však existuje lepší spôsob: náhradný model môžeme iteratívne vylepšovať. Môžeme si zvoliť prvý bod (alebo množinu bodov) rovnomerne náhodne, zostaviť okolo nich model a potom využiť informácie z neho na výber bodov, ktoré použijeme v ďalšom kroku. Začnime tým, že si vytvoríme gaussovský proces – budeme sa usilovať minimalizovať funkciu $\sin([x-2.5]^2)$.



In [ ]:
def func(x):
    return np.sin((x - 2.5) ** 2)

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)

x = np.linspace(0, 5, 100)
plot_distro_func_data(x, gp, func=func)

Čiarkovaná čiara zodpovedá našej účelovej funkcii. Hrubá čierna čiara predstavuje aktuálnu strednú hodnotu predikovanú naším GP. Ako zvyčajne, vyfarbená plocha zodpovedá intervalu $\pm3\sigma$ od strednej hodnoty.

Keďže sme zatiaľ nezískali žiadne pozorovania, máme len apriórne rozdelenie. Nemáme dôvod myslieť si, že tá alebo oná oblasť má vyššiu pravdepodobnosť, že obsahuje minimum. Vyberme si teda počiatočný bod rovnomerne náhodne, ako sme povedali.



In [ ]:
X = [np.random.uniform(0, 5)]
Y = [func(xx) for xx in X]

gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
plot_distro_func_data(x, gp, func=func, X=X, Y=Y)

Zaujímavé samozrejme je, že akonáhle sme si zvolili prvý bod, už máme k dispozícii pozorovanie, ktoré môže informovať náš výber ďalšieho bodu.

### Kompromis medzi prieskumom a úžitkom

Keď vyberáme, ktoré ďalšie body vyhodnotiť, budeme sa snažiť udržať rovnováhu medzi dvoma podcieľmi:

* **Prieskum:**  Snaha znížiť neurčitosť v oblastiach, ktoré ešte nie sú dostatočne preskúmané – aby sme sa dozvedeli či je pravdepodobné, že by mohli obsahovať minimum alebo ich môžeme vylúčiť.
* **Úžitok:**  Snaha sústrediť sa na oblasti, o ktorých už veríme, že by mohli obsahovať minimum (t.j. o ktorých už máme dôkazy, že tam účelová funkcia nadobúda nízke hodnoty).
To, čo sme práve opísali, sa nazýva **kompromis medzi prieskumom a úžitkom**  (the exploration vs. exploitation trade-off) a dá sa s ním stretnúť aj v iných oblastiach, napr. v učení s odmenou (reinforcement learning).

Krásnou vlastnosťou bayesovských metód ako sú gaussovské procesy je, že pracujeme s celým aposteriórnym rozdelením namiesto toho, aby sme predikovali len jednu najpravdepodobnejšiu hodnotu. Vďaka tomu vieme explicitne sledovať neurčitosť a teda aj priamo udrživať rovnováhu medzi priekumom a úžitkom.

Celkovo teda platí, že sa budeme na jednej strane snažiť dostať čím bližšie k minimu a na druhej strane čo najviac znížiť neurčitosť.

### Optimalizácia

Komponent bayesovskej optimalizácie, ktorý nám pomáha zvoliť nasledujúci bod, sa nazýva **akvizičná funkcia** . Keď už máme akvizičnú funkciu k dispozícii, môžeme na nájdenie jej optima voči náhradnému modelu použiť ľubovoľnú existujúcu optimalizačnú metódu. Jednou z najpopulárnejších akvizičných funkcií je **očakávané zlepšenie**  (expected improvement), ktoré je definované takto [[jones]](#jones):

$$
EI(\mathbf{x}) \equiv \mathbb{E} \left[ \max(f_{\min} - f(\mathbf{x}), 0) \right].
$$
kde $f(\mathbf{x})$ vyjadruje hodnotu náhradnej fukcie v bode $\mathbf{x}$ (náhodná premenná), preto potrebujeme pracovať s očakávaním; $f_{\min}$ je najlepšia (najnižšia) hodnota, ktorú sa zatiaľ podarilo nájsť; a zlepšenie musí byť nezáporné, preto oprátor $\max$.

Očakávané zlepšenie teda vyjadruje presne to, čo napovedá jeho názov: očakávanú hodnotu zlepšenia; to znamená, rozdiel medzi najlepšou doteraz nájdenou hodnotou a hodnotou v bode $\mathbf{x}$. Ak $f(\mathbf{x})$ je gaussovké, t.j.:
$f(\mathbf{x}) \sim \mathcal{N}(\mu(\mathbf{x}), \sigma(\mathbf{x}))$
(čo je v prípade gaussovských procesov samozrejme pravda), očakávané zlepšenie sa dá vyjadriť aj pohodlnejším spôsobom [[jones]](#jones):

$$
EI(\mathbf{x}) = \left(
    f_{\min} - \mu(\mathbf{x})
\right)\;
\Phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x})
    }{
        \sigma(\mathbf{x})
    }
\right)
+ \sigma(\mathbf{x}) \phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x})
    }{
        \sigma(\mathbf{x})
    }
\right)
$$
kde $\phi$ je gaussovská hustota pravdepodobnosti (PDF) a $\Phi$ je gaussovská (kumulatívna) distribučná funkcia (CDF).

Aby sme získali väčšiu kontrolu nad kompromisom medzi prieskumom a úžitkom, môžeme od rozdielu $f_{\min} - \mu(\mathbf{x})$  odčítať ešte parameter $\xi$ [[brochu]](#brochu):

$$
EI(\mathbf{x}) = \left(
    f_{\min} - \mu(\mathbf{x}) - \xi
\right)\;
\Phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
+ \sigma(\mathbf{x}) \phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
$$
Rozdiel $f_{\min} - \mu(\mathbf{x})$ je výkonnostný člen, ktorý určuje, aký dobrý je bod $\mathbf{x}$ – o koľko lepší sa javí byť (v zmysle strednej hodnoty $f(\mathbf{x}$) oproti najlepšej doteraz nájdenej hodnote. Zvyšok kritéria máme na to, aby sme do úvahy vzali aj rozptyl (neurčitosť). Odčítaním $\xi$ teda znižujeme váhu výkonnostného členu a tým podporujeme prieskum.

Zdá sa, že vo väčšine prípadov funguje dobre hodnota $\xi = 0.01$, prípadne škálovaná rozptylom signálu. Žíhaním $\xi$ (aby sa podporil prieskum na začiatku a úžitok ku koncu) sa prekvapivo autorom nepodarilo dosiahnuť dobré empirické výsledky [[jones]](#jones).

---
### Úloha 1: Implementujte očakávané zlepšenie

**Implementujte akvizičnú funkciu očakávané zlepšenie prepísaním nižšie uvedených vzorcov do zdrojového kódu v Python-e. Použite rozhranie funkcie predpísané v nasledujúcej bunke.* 

Poznámka: na výpočet gaussovskej PDF a CDF môžete použiť funkcie `norm.pdf` a `norm.cdf`.

$$
EI(\mathbf{x}) = \left(
    f_{\min} - \mu(\mathbf{x}) - \xi
\right)\;
\Phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
+ \sigma(\mathbf{x}) \phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
$$
---


In [ ]:
def expected_improvement(gp, x, fmin, explo_rate=0.01):
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu.reshape(-1); sigma = sigma.reshape(-1)
    

    
    # ---
    
    
    

Napokon použijeme funkciu `expected_improvement` s preddefinovanou pomocnou funkciou `bayes_opti_anim`, ktorá sa postará o optimalizáciu akvizičnej funkcie vo vzťahu ku náhradnej funkcii pomocou metódy LBFGS a následne iteratívne aktualizuje náhradnú funkciu pomocou novozískaných bodov. Výsledky sú prezentované formou animovaného obrázka. Mali by sme vidieť, ako sa účelová funkcia postupne preskúma a nájde sa vhodné minimum.



In [ ]:
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)

fig = plt.figure()
x = np.linspace(0, 5, 100)
frames = lambda: bayes_opti_anim(fig, gp, x, func=func,
               acquisition_func=expected_improvement,
               num_opti_steps=10)

anim = FuncAnimation(fig, func=lambda x: None,
                     interval=200, frames=frames)
HTML(anim.to_jshtml())

### Vizualizácia očakávaného zlepšenia

Aby sme získali o trochu lepšiu intuíciu o tom, čo naša akvizičná funkcia (očakávané zlepšenie) vyjadruje, vizualizujme si ju pre jednoduchý GP podmienený pár dátovými bodmi. Miera prieskumu je nastavená na $\xi=0.01$.



In [ ]:
X = [0.5, 3.5]
Y = [func(xx) for xx in X]

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
fig, axes = plt.subplots(2, 1, sharex=True)

explo_rate=0.01
ac = expected_improvement(gp, x, np.min(Y), explo_rate=explo_rate)

plot_distro_func_data(x, gp, X=X, Y=Y, ax=axes[0])
axes[1].plot(x, ac)
axes[1].set_xlabel("x")
axes[1].set_ylabel("EI(x)")
plt.grid(ls='--')

fig.savefig("output/expected_improvement.svg",
            bbox_inches='tight', pad_inches=0)

### Porozumenie funkcii očakávaného zlepšenia

Teraz sa pokúsme porozumieť trochu lepšie matematike, na ktorej je založené očakávané zlepšenie:

$$
EI(\mathbf{x}) \equiv \mathbb{E} \left[ \max(f_{\min} - f(\mathbf{x}) - \xi, 0) \right].
$$
Prejdime si ju celú postupne.

#### Samotné zlepšenie

Prvou súčasťou vzorca je samotné zlepšenie vyjadrené ako $f_{\min} - f(\mathbf{x})$. Čím väčší tento rozdiel je, tým lepšie je $f(\mathbf{x})$ než predošlá najlepšia hodnota.

#### Ohraničenie zlepšenia zdola

Člen $\max(\text{improvement}, 0)$ ohraničuje zlepšenie zdola na nule. Znamená to v podstate, že ignorujeme všetky prípady, kedy je $f(\mathbf{x})$ v skutočnosti horšie než $f_{\min}$ a považujeme tam hodnotu zlepšenia za 0, hoci inak by bola záporná.

Ako ukazuje nasledujúci obrázok, znamená to, že berieme do úvahy len plochu **pod čiarkovanou červenou čiarou**  (v skutočnosti úplne všetko pod čiarkovanou čiarou, ale vyfarbený interval $\pm 3\sigma$ obsahuje väčšinu pravdepodobnosti). Toto je jediná oblasť, ktorá má akýkoľvek nenulový príspevok.



In [ ]:
#@title -- Expected improvement + visualization of clipping explo_rate=0.01 -- { display-mode: "form" }
X = [0.5, 3.5]
Y = [func(xx) for xx in X]

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
fig, axes = plt.subplots(2, 1, sharex=True)

explo_rate=0.01
ac = expected_improvement(gp, x, np.min(Y), explo_rate=explo_rate)

y_mean, y_std = plot_distro_func_data(x, gp, X=X, Y=Y, ax=axes[0], return_preds=True)
axes[1].plot(x, ac)
axes[1].set_xlabel("x")
axes[1].set_ylabel("EI(x)")
plt.grid(ls='--')

f_min_xi = np.min(Y) - explo_rate
axes[0].axhline(y=f_min_xi, c='r', ls='--', zorder=10, label="$f_{\min}$")
axes[0].fill_between(x, np.minimum(y_mean - 3*y_std, f_min_xi), f_min_xi,
                     color='r', alpha=0.3, zorder=5, label="unclipped area")
axes[0].legend()

fig.savefig("output/expected_improvement_shaded_001.svg",
            bbox_inches='tight', pad_inches=0)

#### Očakávaná hodnota

Zvyšok je v skutočnosti už jednoduchý – berieme očakávanú hodnotu zo zlepšení pod čiarkovanou čiarou, takže budú preferované oblasti, kde náš konfidenčný interval siaha dolu k optimálnejším hodnotám.

#### Miera prieskumu

Aby bolo zrejmejšie, aký vplyv má miera prieskumu $\xi$, zvýšime ju teraz z $0.01$ na $\xi = 0.75$. Ako vidno, čiarková čiara sa tým posúva smerom dolu. Toto pomáha prieskumu, pretože body v rámci tohto pridaného okraja sa budú ignorovať, čo samozrejme nahráva oblastiam s vyššou mierou neurčitosti, kde oblasť $\pm 3 \sigma$ siaha nižšie a preto sa tak veľmi neoreže.



In [ ]:
#@title -- Expected improvement + visualization of clipping with explo_rate=0.75 -- { display-mode: "form" }
X = [0.5, 3.5]
Y = [func(xx) for xx in X]

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
fig, axes = plt.subplots(2, 1, sharex=True)

explo_rate=0.75
ac = expected_improvement(gp, x, np.min(Y), explo_rate=explo_rate)

y_mean, y_std = plot_distro_func_data(x, gp, X=X, Y=Y, ax=axes[0], return_preds=True)
axes[1].plot(x, ac)
axes[1].set_xlabel("x")
axes[1].set_ylabel("EI(x)")
plt.grid(ls='--')

f_min_xi = np.min(Y) - explo_rate
axes[0].axhline(y=f_min_xi, c='r', ls='--', zorder=10, label=r"$f_{\min} - \xi$")
axes[0].fill_between(x, np.minimum(y_mean - 3*y_std, f_min_xi), f_min_xi,
                     color='r', alpha=0.3, zorder=5, label="unclipped area")
axes[0].legend()

fig.savefig("output/expected_improvement_shaded_075.svg",
            bbox_inches='tight', pad_inches=0)

### References

<a id="jones">[jones]</a> Jones, D.R., Schonlau, M. and Welch, W.J., 1998. Efficient global optimization of expensive black-box functions. Journal of Global optimization, 13(4), pp.455-492. [https://link.springer.com/content/pdf/10.1023/A:1008306431147.pdf](https://link.springer.com/content/pdf/10.1023/A:1008306431147.pdf)

<a id="brochu">[brochu]</a> Brochu, E., Cora, V.M. and De Freitas, N., 2010. A tutorial on Bayesian optimization of expensive cost functions, with application to active user modeling and hierarchical reinforcement learning. arXiv preprint arXiv:1012.2599. [https://arxiv.org/abs/1012.2599](https://arxiv.org/abs/1012.2599)

